In [3]:
from NeuronNetwork import NeuronNetwork
from ProbablityEstimation import ProbabilityEstimation
import pandas
import numpy as np
import os
import tensorflow as tf

## On PCA_30 data

In [3]:
filePath = r"X_pca_30.csv"
data = pandas.read_csv(filePath)
dataNP = data.to_numpy()

In [4]:
batchSize = int(dataNP.shape[0] / 100)
cluster = None
prob = None

for i in range(100):
    dataNPSegment = dataNP[i: i+batchSize]
    pe = ProbabilityEstimation(dataNPSegment, 20)
    pe.run()
    currentCluster, _ = pe.getDataSets(1)
    currentCluster = tf.constant(currentCluster)

    if cluster is None:
        cluster = currentCluster
    else:
        cluster = tf.concat([cluster, currentCluster], 0)
        
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [5]:
import gc
gc.collect()

def muLaw(x, mu=255):
    temp = np.sign(x) * np.log(1+mu*np.abs(x)) / np.log(1+mu)
    return ((temp+1)/2 * mu)
def deMuLaw(x, mu=255):
    temp = 2 * (x-mu/2)/mu
    return np.sign(temp) * (1/mu) * ((1+mu)**np.abs(temp) -1)

pe = ProbabilityEstimation(dataNP, cluster.shape[1])
pe.Kmus  = cluster.numpy()
_, prob = pe.getDataSets()
prob = muLaw(prob)
prob = tf.constant(prob)

gc.collect()

0

In [6]:
cluster = tf.reshape(cluster, [cluster.shape[0], cluster.shape[1], 1])
cluster = tf.cast(cluster, tf.float32)
prob = tf.reshape(prob, [prob.shape[0], 1, 1])
prob = tf.cast(prob, tf.float32)

In [8]:
prob = prob / 255
ann = NeuronNetwork(cluster, prob)
ann.train()

Epoch:  1 ; Loss:tf.Tensor(0.24192943, shape=(), dtype=float32)
Epoch:  2 ; Loss:tf.Tensor(0.241928, shape=(), dtype=float32)
Epoch:  3 ; Loss:tf.Tensor(0.24192658, shape=(), dtype=float32)
Epoch:  4 ; Loss:tf.Tensor(0.24192514, shape=(), dtype=float32)
Epoch:  5 ; Loss:tf.Tensor(0.24192367, shape=(), dtype=float32)
Epoch:  6 ; Loss:tf.Tensor(0.24192221, shape=(), dtype=float32)
Epoch:  7 ; Loss:tf.Tensor(0.24192075, shape=(), dtype=float32)
Epoch:  8 ; Loss:tf.Tensor(0.24191923, shape=(), dtype=float32)
Epoch:  9 ; Loss:tf.Tensor(0.24191773, shape=(), dtype=float32)
Epoch:  10 ; Loss:tf.Tensor(0.2419162, shape=(), dtype=float32)
Epoch:  11 ; Loss:tf.Tensor(0.24191466, shape=(), dtype=float32)
Epoch:  12 ; Loss:tf.Tensor(0.24191311, shape=(), dtype=float32)
Epoch:  13 ; Loss:tf.Tensor(0.24191155, shape=(), dtype=float32)
Epoch:  14 ; Loss:tf.Tensor(0.24190998, shape=(), dtype=float32)
Epoch:  15 ; Loss:tf.Tensor(0.24190837, shape=(), dtype=float32)
Epoch:  16 ; Loss:tf.Tensor(0.2419067

Epoch:  128 ; Loss:tf.Tensor(0.24151815, shape=(), dtype=float32)


In [9]:
evalYPath = r"D:\y.csv"
dataEvalY = pandas.read_csv(evalYPath)
dataEvalY = dataEvalY.to_numpy()
print(dataEvalY.shape)

(590540, 1)


In [10]:
dataOriX = tf.cast(tf.constant(dataNP[:,:,None]), tf.float32)

In [11]:
dataPredictY = ann.predict(dataOriX)

In [29]:
dataPredictMappedY = deMuLaw(dataPredictY * 255).reshape([-1])
pe = ProbabilityEstimation(dataPredictMappedY[:,None], 2)
pe.run()
mus,_ = pe.getDataSets()
dataPredictMappedY = dataPredictMappedY.reshape([-1]) < [np.mean(mus)]

In [30]:
dataEvalY = np.reshape(dataEvalY, [-1])

In [31]:
correctCount = np.sum(dataPredictMappedY == dataEvalY)

In [32]:
accuracy = correctCount / dataEvalY.shape[0]

In [33]:
from sklearn.metrics import roc_auc_score
print('Accuracy: ', accuracy)
print('AUC: ', roc_auc_score(dataEvalY, dataPredictMappedY))

Accuracy:  0.962590171707251
AUC:  0.5005885369556662


## One raw data

In [4]:
filePath = r"D:\X.csv"
data = pandas.read_csv(filePath)
dataNP = data.to_numpy()

batchSize = int(dataNP.shape[0] / 100)
cluster = None
prob = None

for i in range(100):
    dataNPSegment = dataNP[i: i+batchSize]
    pe = ProbabilityEstimation(dataNPSegment, 20)
    pe.run()
    currentCluster, _ = pe.getDataSets(1)
    currentCluster = tf.constant(currentCluster)

    if cluster is None:
        cluster = currentCluster
    else:
        cluster = tf.concat([cluster, currentCluster], 0)
        
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [8]:
import gc
gc.collect()

479

In [7]:
gc.collect()

pe = ProbabilityEstimation(dataNP, cluster.shape[1])
pe.Kmus  = cluster.numpy()
_, prob = pe.getDataSets()
prob = muLaw(prob)
prob = tf.constant(prob)

gc.collect()

NameError: name 'muLaw' is not defined

In [9]:
def muLaw(x, mu=255):
    temp = np.sign(x) * np.log(1+mu*np.abs(x)) / np.log(1+mu)
    return ((temp+1)/2 * mu)
def deMuLaw(x, mu=255):
    temp = 2 * (x-mu/2)/mu
    return np.sign(temp) * (1/mu) * ((1+mu)**np.abs(temp) -1)

prob = muLaw(prob)
prob = tf.constant(prob)

gc.collect()

0

In [10]:
cluster = tf.reshape(cluster, [cluster.shape[0], cluster.shape[1], 1])
cluster = tf.cast(cluster, tf.float32)
prob = tf.reshape(prob, [prob.shape[0], 1, 1])
prob = tf.cast(prob, tf.float32)

In [11]:
prob = prob / 255
ann = NeuronNetwork(cluster, prob)
ann.train()

Epoch:  1 ; Loss:tf.Tensor(0.17643099, shape=(), dtype=float32)
Epoch:  2 ; Loss:tf.Tensor(0.14298393, shape=(), dtype=float32)
Epoch:  3 ; Loss:tf.Tensor(0.13288258, shape=(), dtype=float32)
Epoch:  4 ; Loss:tf.Tensor(0.14642225, shape=(), dtype=float32)
Epoch:  5 ; Loss:tf.Tensor(0.14346656, shape=(), dtype=float32)
Epoch:  6 ; Loss:tf.Tensor(0.1531073, shape=(), dtype=float32)
Epoch:  7 ; Loss:tf.Tensor(0.13316706, shape=(), dtype=float32)
Epoch:  8 ; Loss:tf.Tensor(0.13742068, shape=(), dtype=float32)
Epoch:  9 ; Loss:tf.Tensor(0.13065656, shape=(), dtype=float32)
Epoch:  10 ; Loss:tf.Tensor(0.12512001, shape=(), dtype=float32)
Epoch:  11 ; Loss:tf.Tensor(0.121760026, shape=(), dtype=float32)
Epoch:  12 ; Loss:tf.Tensor(0.11835939, shape=(), dtype=float32)
Epoch:  13 ; Loss:tf.Tensor(0.115070075, shape=(), dtype=float32)
Epoch:  14 ; Loss:tf.Tensor(0.106517, shape=(), dtype=float32)
Epoch:  15 ; Loss:tf.Tensor(0.10913654, shape=(), dtype=float32)
Epoch:  16 ; Loss:tf.Tensor(0.10561

In [12]:
evalYPath = r"D:\y.csv"
dataEvalY = pandas.read_csv(evalYPath)
dataEvalY = dataEvalY.to_numpy()
dataEvalY = np.reshape(dataEvalY, [-1])

In [13]:
dataOriX = tf.cast(tf.constant(dataNP[:,:,None]), tf.float32)
dataPredictY = ann.predict(dataOriX)

In [14]:
dataPredictMappedY = deMuLaw(dataPredictY * 255).reshape([-1])
pe = ProbabilityEstimation(dataPredictMappedY[:,None], 2)
pe.run()
mus,_ = pe.getDataSets()
dataPredictMappedY = dataPredictMappedY.reshape([-1]) < [np.mean(mus)]

In [15]:
dataEvalY = np.reshape(dataEvalY, [-1])

In [16]:
correctCount = np.sum(dataPredictMappedY == dataEvalY)

In [17]:
accuracy = correctCount / dataEvalY.shape[0]
from sklearn.metrics import roc_auc_score
print('Accuracy: ', accuracy)
print('AUC: ', roc_auc_score(dataEvalY, dataPredictMappedY))

Accuracy:  0.061729942086903515
AUC:  0.5053194560119841
